# A step-by-step tutorial of pydocker usage
## Main difference between gpu and non-gpu is a extra parameter 'devices' in docker.run()

In [3]:
from dsd.sys.docker import PyDocker
from io import BytesIO
import os

In [4]:
docker = PyDocker()

init pydocker


In [5]:
PyDocker

<class dsd.sys.docker.pydocker.PyDocker at 0x7f0b18054f58>

# 1. Build
## default dockerfile name is $Dockerfile$
Dockerfile:
```
# For Test
FROM ubuntu:14.04
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999
```
## returns: build logs

In [8]:
imageTag = 'joe/buildgputest:cuda7.0'
response = docker.build(tag=imageTag, dockerfilePath='../docker/test_gpu')
if response:
    for line in response:
        print(line)
else:
    print('build failed')

# For Test
FROM guoq/imxnet:cuda-7.0
MAINTAINER zy <zy3381@gmail.com>
EXPOSE 9999

('Connection aborted.', error(13, 'Permission denied'))


ConnectionError: ('Connection aborted.', error(13, 'Permission denied'))

# 2. list images

In [5]:
pses = docker.images()
for line in pses:
    print line['RepoTags']

joe/buildgputest:cuda7.0
sample:nvidia-smi
sample:vectorAdd
sample:nbody
sample:matrixMulCUBLAS
sample:deviceQuery
sample:bandwidthTest
cuda:7.5
cuda:7.5-runtime
wlt_torch:latest
nvidia/cuda:7.0
nvidia/cuda:latest
joe/test1:latest
wzz/lasagne:latest
kaixhin/cuda-theano:7.0
shawn/cuda-mxnet:422
<none>:<none>
cuda-mxnet:20160421
kaixhin/cudnn:7.0
wangjianyong/mxnet_jupyter:latest
caffe:latest
guoq/imxnet:cuda-7.0
<none>:<none>
ubuntu:14.04
hello-world:latest
kaixhin/cuda-digits:latest
kaixhin/cuda:7.0


# 3. run
## $imageTag$ comes from step 1
## params:
## name='myUbuntuTest',
==============================================================
## ports_dict={1234:8888, 5678:7777}, similar to 'docker -p'
### [hostPort:containerPort],   type:dict
## ports_list=[9999], similar to 'docker -P'
### dynamic assign a port on host 
#### net stat: *0.0.0.0:5678->7777/tcp, 0.0.0.0:1234->8888/tcp, 0.0.0.0:32770->9999/tcp*
==============================================================
## volumes={'/home/zhouyao/':'/mnt/homeOfJoe'},   type:dict
#### volume in host: */home/zhouyao/*, volume in container: */mnt/homeOfJoe*
==============================================================
## GPU devices: specified four gpu cards(0-3),  type:list

#### tests below all successed in mxnet 

```
import mxnet as mx
a1 = mx.ndarray.ones([1,1], ctx=mx.gpu(0))
a2 = mx.ndarray.ones([1,1], ctx=mx.gpu(1))
a3 = mx.ndarray.ones([1,1], ctx=mx.gpu(2))
a4 = mx.ndarray.ones([1,1], ctx=mx.gpu(3))
```

In [6]:
container = docker.run(image=imageTag, 
           detach=True, 
           stdin_open=True, 
           tty=True, 
           command='bash', 
           name='myGpuTest',
           user=None,
                        
           ports_dict={1234:8888, 5678:7777}, 
           ports_list=[9999],
                       
           volumes={'/home/zhouyao/':'/mnt/homeOfJoe'},
                       
           devices=[0,1,2,3]
           )
if container is not None:
    print 'container is runing with id:'
    print container.get('Id')
else:
    print 'create container error'

container is runing with id:
2c3c47b55daf6d5db550711c613c99239e25f83478120b0813cc06241b0d1f72


# 4. start
## *container_id* comes from step 3

In [7]:
container_id = container.get('Id')
response = docker.start(container=container_id)
if response:
    print 'container ', container_id, ' started'
else:
    print 'starting container error'

container  2c3c47b55daf6d5db550711c613c99239e25f83478120b0813cc06241b0d1f72  started


# 5. check status

In [8]:
pses = docker.ps()
print 'Runing containers:'
print '##################'
for line in pses:
    print line['image'], line['name']

Runing containers:
##################
joe/buildgputest:cuda7.0 /myGpuTest
ubuntu:14.04 /simpleDQN
ubuntu:14.04 /simple_dqn
wzz/lasagne /wzz_docker
guoq/imxnet:cuda-7.0 /lwb-mxnet
sha256:c832a806ee999c1091fc279fd427bf331e838c57789e21d9d59ed4b98826c377 /wjy-mxnet


# 6. Stop 

In [9]:
container_id = container.get('Id')
response = docker.stop(container=container_id,
                       timeout=0)
if response:
    print 'container', container_id, 'stop succeed'
else:
    print '#######################'
    print 'container stop failed'

container 2c3c47b55daf6d5db550711c613c99239e25f83478120b0813cc06241b0d1f72 stop succeed


# 7. Remove container

In [10]:
container_id = container.get('Id')
response = docker.rm(container=container_id)
if response:
    print 'container ', container_id, 'removed succed'
else:
    print '#######################'
    print 'container removed failed'

container  2c3c47b55daf6d5db550711c613c99239e25f83478120b0813cc06241b0d1f72 removed succed


# 8. Remove image

In [11]:
response = docker.rmi(image=imageTag,
                      force=False,
                      noprune=None)
if response:
    print 'image ', imageTag, 'removed succeed'
else:
    print '#######################'
    print 'image removed failed'

image  joe/buildgputest:cuda7.0 removed succeed
